In [1]:
import moviepy.editor as mpy
import numpy as np
import pandas as pd
import os
import cv2
import re

In [2]:
def remove_outliers(src):
    num_labels,labels,stats,centroids = cv2.connectedComponentsWithStats(src,connectivity=4,ltype=None)
    img = np.zeros((src.shape[0],src.shape[1]),np.uint8) # create a black background of all 0
    for i in range(1,num_labels):
        mask = labels == i # this step is to determine the location of the area through labels, assign labels information to the mask array, and then use the mask array as the index of img array
        if stats[i][4] > 100: 
            img[mask] = 255 # areas larger than 100 shall be painted white, and areas smaller than 100 shall be painted black
        else:
            img[mask] = 0
    return img

In [3]:
def get_contours(frm):
    # convert frame to grayscale
    frm_gray = cv2.cvtColor(frm, cv2.COLOR_BGR2GRAY)
    # remove tiny particels
    frm_out = remove_outliers(frm_gray)
    # convert frame to black and white
    frm_thresh = cv2.threshold(frm_out,0,255,cv2.THRESH_BINARY)[1]
    # extract contours
    contours,_ = cv2.findContours(frm_thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

In [4]:
def get_bbox_coords(clip, t=0):
    x_bbox=y_bbox=w_bbox=h_bbox=0
    # get the frame
    frm = clip.get_frame(t)
    # get the bounding box of the frame
    contours = get_contours(frm)
    # get the coordinates of the bounding box
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if w > 100 and h > 100:
            x_bbox,y_bbox,w_bbox,h_bbox=x,y,w,h
    
    return x_bbox,y_bbox,w_bbox,h_bbox

In [5]:
def crop_and_resize(clip): 
    clip_x, clip_y = clip.size
    bbox = [clip_x,0,clip_y,0]
    average_h = 740
    
    # iterate through frames of the video clip
    for frm in clip.iter_frames():
        contours = get_contours(frm)
        
        # iterate over contours
        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            # save rectangle that contains all boundingboxes inside it
            if x < bbox[0]:
                bbox[0] = x 
            if x+w > bbox[1]:
                bbox[1] = x+w 
            if y < bbox[2]:
                bbox[2] = y 
            if y+h > bbox[3]:
                bbox[3] = y+h 
    
    # crop and resize the clip
    _,_,_,clip_h = get_bbox_coords(clip)
    scaling_factor =  average_h/clip_h
    
    cropped_clip = clip.fx(mpy.vfx.crop, x1=bbox[0],x2=bbox[1],y1=bbox[2],y2=bbox[3])
    resized_clip = cropped_clip.resize(scaling_factor)
    
    #calculate the center of gravity of the human body in the resized clip
    x,y,w,h = get_bbox_coords(resized_clip)
    center = (round((2*x+w)/2),round((2*y+h)/2))
    
    return resized_clip, center

In [6]:
def change_dur(clip, duration):
    rate = clip.fps
    dur = clip.duration
    
    desired_rate = (rate*dur)/duration
    scaling_factor = desired_rate/rate

    # Modify the FPS
    clip = clip.set_fps(clip.fps*scaling_factor)
    # Apply speed up
    clip = clip.fx(mpy.vfx.speedx,scaling_factor)
    
    return clip

In [7]:
def fix_rotation(clip):
    if clip.rotation in (90,270):
            clip = clip.resize(clip.size[::-1])
            clip.rotation = 0
            
    return clip

In [8]:
def create_gesture_combinations(pairings, is_1G=False):
    for ind, pair in enumerate(pairings):
        gestures_left = []
        gestures_right = []
        if is_1G:
            regex_left = rf'({pair[0]})_(f01|p)'
            regex_right = rf'({pair[1]})_(f02|n)'
        else:
            regex_left = rf'({pair[0]}|{pair[1]})_(f01|p)'
            regex_right = rf'({pair[0]}|{pair[1]})_(f02|n)'

        for f in files:
            if re.search(regex_left, f):
                gestures_left.append(f)
            elif re.search(regex_right, f):
                gestures_right.append(f)
        pairings[ind] = [(left,right) for right in gestures_right for left in gestures_left]
        
    return pairings

In [9]:
def determine_interaction_type(left, right):
    if len(left[26:-4]) != len(right[26:-4]):
        return 'HR'
    elif len(left[26:-4]) == 1:
        return 'RR'
    else: 
        return 'HH'

In [10]:
def determine_gesture_type(left, right):
    if left[21:25] == 'gg00':
        return '1G'
    elif left[21:25] != right[21:25]:
        return '2G_DIFF'
    else: 
        return '2G_SAME'

In [11]:
in_dir = 'IndividualStimuli'
files = [os.path.join(in_dir, f) for f in os.listdir(in_dir)]

root_dir = 'DyadStimuli'
sub_dirs = ['1G', '2G_SAME', '2G_DIFF']
sub_sub_dirs = ['HH','HR','RR']

is_existend = os.path.exists(root_dir)
if not is_existend:
    os.mkdir(root_dir)
    for sub_dir in sub_dirs:
        path_sub_dir = os.path.join(root_dir,sub_dir)
        os.mkdir(path_sub_dir)
        for sub_sub_dir in sub_sub_dirs:
            path_sub_sub_dir = os.path.join(root_dir,sub_dir,sub_sub_dir)
            os.mkdir(path_sub_sub_dir)

pairings_2G = [('gg09', 'gg19'), ('gg01', 'gg18'), ('gg10', 'gg11'), ('gg13', 'gg17'), 
                     ('gg08', 'gg20'), ('gg04', 'gg15'), ('gg02', 'gg06'), ('gg03', 'gg07'),]

all_gestures_df = pd.read_excel('GesturesIndividualStimuli.xlsx', 'gestures_combinations')
relevant_gestures_df = all_gestures_df.dropna()
gesture_codes = relevant_gestures_df['GestureCode'].tolist()

pairings_1G = []
for i in gesture_codes[1:]:
    single_gesture = ('gg00', i)
    pairings_1G.append(single_gesture)

combinations_2G = create_gesture_combinations(pairings_2G)
combinations_1G = create_gesture_combinations(pairings_1G, True)
all_combinations = combinations_2G + combinations_1G

for cnt,comb in enumerate(all_combinations[::len(all_combinations)-1]):
    for gestures in comb:
        # Determine the interaction and gesture type of the dyad stimuli to be created
        interaction_type = determine_interaction_type(gestures[0],gestures[1])
        gesture_type = determine_gesture_type(gestures[0],gestures[1])
        
        # Load the video files of the individual stimuli to be combined into a dyad
        gesture_left = mpy.VideoFileClip(gestures[0],has_mask=True)
        gesture_right = mpy.VideoFileClip(gestures[1],has_mask=True).fx(mpy.vfx.mirror_x)

        # Workaround for the moviepy bug causing videos with rotation metadata to be stretched
        gesture_left = fix_rotation(gesture_left)
        gesture_right = fix_rotation(gesture_right)

        # Transform the individual stimuli files
        gesture_left, center_left = crop_and_resize(gesture_left)
        gesture_right, center_right = crop_and_resize(gesture_right)

        # Speed up or slow down video to 3sec
        duration = 3
        gesture_right = change_dur(gesture_right,duration)
        gesture_left = change_dur(gesture_left,duration)

        # Load the background image
        background = mpy.ImageClip('background_dyads.png',duration=duration)

        # Create the video composition
        dyad = mpy.CompositeVideoClip([background, 
                                       gesture_left.set_position((660-center_left[0],background.size[1]-gesture_left.size[1]-170)), 
                                       gesture_right.set_position((1260-center_right[0],background.size[1]-gesture_right.size[1]-170))
                                      ])
        # Make the video grayscale
        dyad = dyad.fx(mpy.vfx.blackwhite)

        # Write the output video file
        out_path = os.path.join(root_dir,gesture_type,interaction_type,f'DS_l{gestures[0][21:-4]}_r{gestures[1][21:-4]}.mp4')
        dyad.write_videofile(out_path,fps=25,audio=False)

Moviepy - Building video DyadStimuli/2G_SAME/HR/DS_lgg09_f01_rgg09_n.mp4.
Moviepy - Writing video DyadStimuli/2G_SAME/HR/DS_lgg09_f01_rgg09_n.mp4



Moviepy - Done !
Moviepy - video ready DyadStimuli/2G_SAME/HR/DS_lgg09_f01_rgg09_n.mp4
Moviepy - Building video DyadStimuli/2G_SAME/RR/DS_lgg09_p_rgg09_n.mp4.
Moviepy - Writing video DyadStimuli/2G_SAME/RR/DS_lgg09_p_rgg09_n.mp4



Moviepy - Done !
Moviepy - video ready DyadStimuli/2G_SAME/RR/DS_lgg09_p_rgg09_n.mp4
Moviepy - Building video DyadStimuli/2G_DIFF/HR/DS_lgg19_f01_rgg09_n.mp4.
Moviepy - Writing video DyadStimuli/2G_DIFF/HR/DS_lgg19_f01_rgg09_n.mp4



Moviepy - Done !
Moviepy - video ready DyadStimuli/2G_DIFF/HR/DS_lgg19_f01_rgg09_n.mp4
Moviepy - Building video DyadStimuli/2G_DIFF/RR/DS_lgg19_p_rgg09_n.mp4.
Moviepy - Writing video DyadStimuli/2G_DIFF/RR/DS_lgg19_p_rgg09_n.mp4



Moviepy - Done !
Moviepy - video ready DyadStimuli/2G_DIFF/RR/DS_lgg19_p_rgg09_n.mp4
Moviepy - Building video DyadStimuli/2G_DIFF/HH/DS_lgg09_f01_rgg19_f02.mp4.
Moviepy - Writing video DyadStimuli/2G_DIFF/HH/DS_lgg09_f01_rgg19_f02.mp4



...

Moviepy - Done !
Moviepy - video ready DyadStimuli/2G_SAME/RR/DS_lgg19_p_rgg19_n.mp4
Moviepy - Building video DyadStimuli/1G/HH/DS_lgg00_f01_rgg20_f02.mp4.
Moviepy - Writing video DyadStimuli/1G/HH/DS_lgg00_f01_rgg20_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadStimuli/1G/HH/DS_lgg00_f01_rgg20_f02.mp4
Moviepy - Building video DyadStimuli/1G/HR/DS_lgg00_p_rgg20_f02.mp4.
Moviepy - Writing video DyadStimuli/1G/HR/DS_lgg00_p_rgg20_f02.mp4



Moviepy - Done !
Moviepy - video ready DyadStimuli/1G/HR/DS_lgg00_p_rgg20_f02.mp4
Moviepy - Building video DyadStimuli/1G/HR/DS_lgg00_f01_rgg20_n.mp4.
Moviepy - Writing video DyadStimuli/1G/HR/DS_lgg00_f01_rgg20_n.mp4



Moviepy - Done !
Moviepy - video ready DyadStimuli/1G/HR/DS_lgg00_f01_rgg20_n.mp4
Moviepy - Building video DyadStimuli/1G/RR/DS_lgg00_p_rgg20_n.mp4.
Moviepy - Writing video DyadStimuli/1G/RR/DS_lgg00_p_rgg20_n.mp4



Moviepy - Done !
Moviepy - video ready DyadStimuli/1G/RR/DS_lgg00_p_rgg20_n.mp4
